In [4]:
#Importing the required libraries
import cv2
import os
import sys
import face_recognition
import numpy as np
from PyEmotion import *

In [8]:
#Python program to detect face, its features,recognize it and detect facial emotion in real-time using webcam.

print("***FACE DETECTION, FACE RECOGNITION & EMOTION RECOGNITION***")
print()
print("***MAIN MENU***")
print()
print("1.Face Detection & Feature Recognition")
print("2.Face Recognition")
print("3.Emotion Recognition")
print("4.Exit")
print()


c = int(input("Enter your choice: "))

while(1):
    if c==1:
        print("Welcome to face detection & feature recognition!")
        print()
        print("Please wait while the camera is being configured.")
        print()
        
        #Method to draw boundary around detected features 
        def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text):
            
            #For detection, conversion grayscale is needed 
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            #Generic function which detects features once classifier is passed.
            #By Scaling factor we can scale our classifier to detect a small or large face 
            #depending upon the face's proximity to the camera.
            #Minimum neigbours basically means how many minimum features are to be detected so that the thing under 
            #observation can be classified as a face or not.
            #It returns a list of features.
            features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
            
            #List which holds coordinates for face
            coords = []
            for (x, y, w, h) in features:
                
                #Drawing a rectangle around detected feature
                cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
                
                #Labelling the detected feature 
                cv2.putText(img, text, (x,y-4), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
                
                #Update the coordinates
                coords = [x, y, w, h]
    
            #return the updated image(frame)
            return coords
        
        #Function which takes the frame and classifiers and returns an updated frame 
        #which shows all the detected features along with the face
        def detect(img, faceCascade, eyesCascade, noseCascade, mouthCascade):
            
            #creating a dictionary for colours so as to easily access them when required
            color = {"blue":(255,0,0), "red":(0,0,255), "green":(0,255,0), "white":(255,255,255)}
            
            coords = draw_boundary(img, faceCascade, 1.1, 10, color['blue'], "Face")
            
            #To check if face has been detected or not, and if it has been detected,
            #then crop the frame to our region of interest, update it, and then
            #use this updated frame to draw rectangles around the detected features
            #by passing respective classifiers and other parameters using draw_boundary function
            if len(coords) == 4:
                #updated the coordinates, cropped the video frame to region of interest
                roi_img = img[coords[1]:coords[1]+coords[3], coords[0]:coords[0]+coords[2]]
                
                #Detecting eyes,nose and mouth in our region of interest
                coords = draw_boundary(roi_img, eyesCascade, 1.1, 14, color['red'], "Eyes")
                coords = draw_boundary(roi_img, noseCascade, 1.1, 5, color['green'], "Nose")
                coords = draw_boundary(roi_img, mouthCascade, 1.1, 20, color['white'], "Mouth")
    
            #returning the updated frame
            return img
        
        #Loading and passing the classifiers for face and feature detection
        faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
        eyesCascade = cv2.CascadeClassifier("haarcascade_eye.xml")
        noseCascade = cv2.CascadeClassifier("Nariz.xml")
        mouthCascade = cv2.CascadeClassifier("Mouth.xml")
        
        # Get a reference to webcam 0 (the default one)  
        video_capture = cv2.VideoCapture(0)
        
        #Infinite loop to detect face and its features 
        while True:
            #Captures a frame from the video
            _, img = video_capture.read()
            
            #Detects face and its features in that frame
            img = detect(img, faceCascade, eyesCascade, noseCascade, mouthCascade)
            
            #Shows the updated frame 
            cv2.imshow("Face Detection", img)
            
            #Press the button 'q' to exit the video
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        #Releases the webcam & destroys all the windows
        video_capture.release()
        cv2.destroyAllWindows()
        print("Thank you for using face detection!")
        print()
        break
        
    elif c==2:
        print("Welcome to face recognition")
        print()
        print("Please wait while the camera is being configured.")
        print()
        
        # Get a reference to webcam 0 (the default one)
        video_capture = cv2.VideoCapture(0)

        # Loading a sample picture and learning how to recognize it.
        modi_image = face_recognition.load_image_file("modi.jfif")
        modi_face_encoding = face_recognition.face_encodings(modi_image)[0]

        # Loading a second sample picture and learning how to recognize it.
        zuckerberg_image = face_recognition.load_image_file("zuckerberg.jfif")
        zuckerberg_face_encoding = face_recognition.face_encodings(zuckerberg_image)[0]


        # Create arrays of known face encodings and their names
        known_face_encodings = [
            modi_face_encoding,
            zuckerberg_face_encoding,
        ]
        known_face_names = [
            "Narendra Modi",
            "Mark Zuckerberg"
        ]

        # Creating lists 
        face_locations = []
        face_encodings = []
        face_names = []
        process_this_frame = True
        
        while True:
            # Grabbing a single frame of video
            ret, frame = video_capture.read()

            # Resizing frame of video to 1/4 size for faster face recognition processing
            small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

            # Converting the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
            rgb_small_frame = small_frame[:, :, ::-1]

            # Only processing every other frame of video to save time
            if process_this_frame:
                # Finding all the faces and face encodings in the current frame of video
                face_locations = face_recognition.face_locations(rgb_small_frame)
                face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

                face_names = []
                for face_encoding in face_encodings:
                    # Seeing if the face is a match for the known face(s) by comparing the list
                    # of face encodings against a candidate encoding to see if they match, returns
                    # a list of True/false values
                    matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                    name = "Unknown"

                    #If a match was found in known_face_encodings, just using the first one.
                    # if True in matches:
                    #     first_match_index = matches.index(True)
                    #     name = known_face_names[first_match_index]

                    # Or instead, using the known face with the smallest(min) distance(euclidian) to the new face.
                    # Given a list of face encodings, compare them to a known face encoding & get a
                    # euclidean distance for each comparision face. This tells us how similar the faces are.
                    face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                    best_match_index = np.argmin(face_distances)
                    if matches[best_match_index]:
                        name = known_face_names[best_match_index]

                    face_names.append(name)

            process_this_frame = not process_this_frame
            
            # Display the results
            for (top, right, bottom, left), name in zip(face_locations, face_names):
                # Scale back up face locations since the frame we detected in was scaled to 1/4 its size
                top *= 4
                right *= 4
                bottom *= 4
                left *= 4

                # Draw a box around the face
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

                # Draw a label with a name below the face
                cv2.rectangle(frame, (left, bottom - 25), (right, bottom), (0, 0, 255), cv2.FILLED)
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.6, (255, 255, 255), 1, cv2.LINE_AA)

            # Display the resulting image
            cv2.imshow('Face Recognition', frame)

            # To quit, hit 'q' on the keyboard 
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            
        # Release handle to the webcam
        video_capture.release()
        cv2.destroyAllWindows()
        break
        
    elif c==3:
        print("Welcome to Emotion Recognition!")
        print()

        #inbuilt function inside the library
        PyEmotion()
        
        print("Please wait while the camera is being configured.")
        print()
        
        #To detect face(init PyEmotion function)
        #The __init__ method can be called when an object is created from the class, 
        #and access is required to initialize the attributes of the class.
        er = DetectFace(device='cpu', gpu_id=0)

        # Opening default camera
        cap = cv2.VideoCapture(0)
        
        #This code initiates an infinite loop (to be broken later by a break statement), 
        #where we have ret and frame being defined as the cap.read(). 
        #Basically, ret is a boolean regarding whether or not there was a return at all, 
        #at the frame is each frame that is returned. 
        #If there is no frame, we won't get an error, we will get None.
        while(True):
            
            #Reads a single frame from the video
            ret, frame = cap.read()
            
            #Predicts emotion in that frame where 
            #frame->returns the frame with emotion & 
            #emotion->returns the frame of the emotion
            frame, emotion = er.predict_emotion(frame)
            
            #Displays the predicted emotion
            cv2.imshow('Emotion detection', frame)
            
            #To quit, enter the 'q' button in keyboard
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        #Releases the webcam & destroys all the windows
        cap.release()
        cv2.destroyAllWindows()
        print("Thank you for using emotion recognition!")
        print()
        break
        
    else:
        print("Thank you!")
        print()
        break       
        
print("***END OF PROGRAM***")

***FACE DETECTION, FACE RECOGNITION & EMOTION RECOGNITION***

***MAIN MENU***

1.Face Detection & Feature Recognition
2.Face Recognition
3.Emotion Recognition
4.Exit

Enter your choice: 3
Welcome to Emotion Recognition!

 ____          _____                    _    _               
|  _ \  _   _ | ____| _ __ ___    ___  | |_ (_)  ___   _ __  
| |_) || | | ||  _|  | '_ ` _ \  / _ \ | __|| | / _ \ | '_ \ 
|  __/ | |_| || |___ | | | | | || (_) || |_ | || (_) || | | |
|_|     \__, ||_____||_| |_| |_| \___/  \__||_| \___/ |_| |_|
        |___/                                                

Welcome to PyEmotion 
Please wait while the camera is being configured.

Thank you for using emotion recognition!

***END OF PROGRAM***
